# Phân Loại Bệnh Lá Đậu (Bean Leaf Disease Classification)

## Giới thiệu
Notebook này xây dựng mô hình CNN để phân loại bệnh trên lá đậu từ hình ảnh.

### Các loại bệnh được phân loại:
- **Healthy** - Lá khỏe mạnh
- **Angular Leaf Spot** - Bệnh đốm góc lá
- **Bean Rust** - Bệnh gỉ sắt đậu

### Kỹ thuật chống overfitting:
1. Data Augmentation mạnh
2. Dropout layers
3. Batch Normalization
4. Early Stopping
5. Learning Rate Scheduling
6. L2 Regularization
7. Transfer Learning

---

## 1. Import thư viện

In [ ]:
# Import các thư viện cần thiết
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# TensorFlow và Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, 
    BatchNormalization, GlobalAveragePooling2D, Input
)
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0, DenseNet121
from tensorflow.keras.regularizers import l2

# Sklearn cho đánh giá
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Kiểm tra GPU
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

# Seed để tái lập kết quả
np.random.seed(42)
tf.random.set_seed(42)

## 2. Cấu hình và hằng số

In [ ]:
# Cấu hình
IMG_SIZE = (224, 224)  # Kích thước ảnh đầu vào
BATCH_SIZE = 32  # Kích thước batch
EPOCHS = 50  # Số epoch tối đa

# Các class
CLASSES = ['healthy', 'angular_leaf_spot', 'bean_rust']
NUM_CLASSES = len(CLASSES)

# Mapping nhãn
CATEGORY_MAP = {0: 'healthy', 1: 'angular_leaf_spot', 2: 'bean_rust'}
CLASS_NAMES_VN = {
    'healthy': 'Lá khỏe mạnh',
    'angular_leaf_spot': 'Bệnh đốm góc lá', 
    'bean_rust': 'Bệnh gỉ sắt đậu'
}

# Đường dẫn dữ liệu
BASE_DIR = os.getcwd()
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
VAL_DIR = os.path.join(BASE_DIR, 'val')
MODEL_DIR = os.path.join(BASE_DIR, 'models')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')

# Tạo thư mục nếu chưa có
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Train directory: {TRAIN_DIR}")
print(f"Validation directory: {VAL_DIR}")
print(f"Model directory: {MODEL_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

## 3. Khám phá dữ liệu (EDA)

In [ ]:
def count_images(data_path):
    """Đếm số ảnh trong mỗi class"""
    counts = {}
    for cls in CLASSES:
        cls_path = os.path.join(data_path, cls)
        if os.path.exists(cls_path):
            imgs = [f for f in os.listdir(cls_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
            counts[cls] = len(imgs)
    return counts

# Đếm số ảnh
train_counts = count_images(TRAIN_DIR)
val_counts = count_images(VAL_DIR)

print("=" * 50)
print("THỐNG KÊ DỮ LIỆU")
print("=" * 50)
print(f"\nTổng ảnh training: {sum(train_counts.values())}")
print(f"Tổng ảnh validation: {sum(val_counts.values())}")
print(f"\nPhân bố Training:")
for cls, cnt in train_counts.items():
    print(f"  - {cls}: {cnt} ảnh ({cnt/sum(train_counts.values())*100:.1f}%)")
print(f"\nPhân bố Validation:")
for cls, cnt in val_counts.items():
    print(f"  - {cls}: {cnt} ảnh ({cnt/sum(val_counts.values())*100:.1f}%)")

In [ ]:
# Vẽ biểu đồ phân bố
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Biểu đồ Training
colors = ['#2ecc71', '#e74c3c', '#f39c12']
axes[0].bar(train_counts.keys(), train_counts.values(), color=colors)
axes[0].set_title('Phân bố dữ liệu Training', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Loại bệnh')
axes[0].set_ylabel('Số lượng ảnh')
for i, (cls, cnt) in enumerate(train_counts.items()):
    axes[0].text(i, cnt + 5, str(cnt), ha='center', fontweight='bold')

# Biểu đồ Validation
axes[1].bar(val_counts.keys(), val_counts.values(), color=colors)
axes[1].set_title('Phân bố dữ liệu Validation', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Loại bệnh')
axes[1].set_ylabel('Số lượng ảnh')
for i, (cls, cnt) in enumerate(val_counts.items()):
    axes[1].text(i, cnt + 1, str(cnt), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'data_distribution.png'), dpi=150)
plt.show()

In [ ]:
# Hiển thị ảnh mẫu từ mỗi class
def show_sample_images(data_path, n=4):
    """Hiển thị ảnh mẫu từ mỗi class"""
    fig, axes = plt.subplots(len(CLASSES), n, figsize=(16, 12))
    
    for i, cls in enumerate(CLASSES):
        cls_path = os.path.join(data_path, cls)
        if os.path.exists(cls_path):
            imgs = [f for f in os.listdir(cls_path) if f.endswith(('.jpg', '.jpeg', '.png'))][:n]
            for j, img_name in enumerate(imgs):
                img_path = os.path.join(cls_path, img_name)
                img = Image.open(img_path)
                axes[i, j].imshow(img)
                if j == 0:
                    axes[i, j].set_ylabel(f"{cls}\n({CLASS_NAMES_VN[cls]})", fontsize=10, fontweight='bold')
                axes[i, j].axis('off')
    
    plt.suptitle('Ảnh mẫu từ mỗi loại bệnh', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'sample_images.png'), dpi=150)
    plt.show()

show_sample_images(TRAIN_DIR)

## 4. Tiền xử lý và Data Augmentation

### Kỹ thuật chống overfitting:
- **Rotation**: Xoay ảnh ngẫu nhiên
- **Zoom**: Phóng to/thu nhỏ
- **Flip**: Lật ngang/dọc
- **Shift**: Dịch chuyển
- **Shear**: Biến dạng

In [ ]:
# Data Augmentation mạnh cho training
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Chuẩn hóa [0,1]
    rotation_range=30,           # Xoay ngẫu nhiên 0-30 độ
    width_shift_range=0.2,       # Dịch ngang 20%
    height_shift_range=0.2,      # Dịch dọc 20%
    shear_range=0.2,             # Biến dạng shear
    zoom_range=0.3,              # Zoom 0.7-1.3
    horizontal_flip=True,        # Lật ngang
    vertical_flip=True,          # Lật dọc
    brightness_range=[0.8, 1.2], # Độ sáng
    fill_mode='nearest'          # Điền pixel
)

# Chỉ rescale cho validation (không augmentation)
val_datagen = ImageDataGenerator(rescale=1./255)

# Tạo data generators
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=CLASSES,
    shuffle=True,
    seed=42
)

val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=CLASSES,
    shuffle=False
)

print(f"\nTraining samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")
print(f"Class indices: {train_generator.class_indices}")

In [ ]:
# Hiển thị ảnh sau khi augmentation
def show_augmented_images():
    """Hiển thị ảnh gốc và sau augmentation"""
    # Lấy một batch
    sample_batch, _ = next(train_generator)
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    for i in range(8):
        ax = axes[i//4, i%4]
        ax.imshow(sample_batch[i])
        ax.axis('off')
        ax.set_title(f'Augmented Image {i+1}')
    
    plt.suptitle('Ảnh sau Data Augmentation', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'augmented_samples.png'), dpi=150)
    plt.show()

show_augmented_images()

## 5. Xây dựng mô hình CNN

### Mô hình 1: CNN cơ bản với các kỹ thuật chống overfitting

In [ ]:
def build_cnn_model(input_shape=(224, 224, 3), num_classes=NUM_CLASSES):
    """
    Xây dựng mô hình CNN với các kỹ thuật chống overfitting:
    - BatchNormalization: Chuẩn hóa đầu ra mỗi layer
    - Dropout: Ngắt ngẫu nhiên các neuron
    - L2 Regularization: Phạt trọng số lớn
    - GlobalAveragePooling: Giảm số lượng tham số
    """
    model = Sequential([
        # Block 1
        Conv2D(32, (3, 3), activation='relu', padding='same', 
               kernel_regularizer=l2(0.001), input_shape=input_shape),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        Dropout(0.25),
        
        # Block 2
        Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        Dropout(0.25),
        
        # Block 3
        Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        Dropout(0.25),
        
        # Block 4
        Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        Dropout(0.25),
        
        # Global Average Pooling thay vì Flatten (giảm overfitting)
        GlobalAveragePooling2D(),
        
        # Fully connected layers
        Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.5),
        
        # Output layer
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# Tạo model CNN
cnn_model = build_cnn_model()
cnn_model.summary()

### Mô hình 2: Transfer Learning với MobileNetV2

In [ ]:
def build_mobilenet_model(input_shape=(224, 224, 3), num_classes=NUM_CLASSES):
    """
    Xây dựng mô hình Transfer Learning với MobileNetV2.
    - Sử dụng weights đã pre-trained trên ImageNet
    - Đóng băng base model ban đầu
    - Thêm custom layers phía trên
    """
    # Load MobileNetV2 pre-trained (không bao gồm top layers)
    base_model = MobileNetV2(
        weights='imagenet', 
        include_top=False, 
        input_shape=input_shape
    )
    
    # Đóng băng base model
    base_model.trainable = False
    
    # Xây dựng model
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        BatchNormalization(),
        Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    return model, base_model

# Tạo MobileNet model
mobilenet_model, mobilenet_base = build_mobilenet_model()
mobilenet_model.summary()

### Mô hình 3: Transfer Learning với EfficientNetB0 (Recommended cho accuracy cao)

In [ ]:
def build_efficientnet_model(input_shape=(224, 224, 3), num_classes=NUM_CLASSES):
    """
    Xây dựng mô hình Transfer Learning với EfficientNetB0.
    EfficientNet thường cho accuracy cao nhất.
    """
    # Load EfficientNetB0 pre-trained
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Đóng băng base model
    base_model.trainable = False
    
    # Xây dựng model
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        BatchNormalization(),
        Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.4),
        Dense(num_classes, activation='softmax')
    ])
    
    return model, base_model

# Tạo EfficientNet model
efficientnet_model, efficientnet_base = build_efficientnet_model()
efficientnet_model.summary()

## 6. Cấu hình Training

### Callbacks để chống overfitting và tối ưu training

In [ ]:
def get_callbacks(model_name='model'):
    """
    Tạo callbacks cho training:
    - EarlyStopping: Dừng sớm khi không cải thiện
    - ModelCheckpoint: Lưu model tốt nhất
    - ReduceLROnPlateau: Giảm learning rate khi plateau
    """
    callbacks = [
        # Dừng sớm nếu val_loss không giảm trong 10 epoch
        EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=1
        ),
        
        # Lưu model có val_accuracy cao nhất
        ModelCheckpoint(
            filepath=os.path.join(MODEL_DIR, f'{model_name}_best.keras'),
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        ),
        
        # Giảm learning rate khi val_loss không giảm
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-7,
            verbose=1
        )
    ]
    return callbacks

print("Callbacks đã được cấu hình!")

## 7. Huấn luyện mô hình

### Train MobileNetV2 (Recommended - Cân bằng accuracy và tốc độ)

In [ ]:
# Compile MobileNet model
mobilenet_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("="*50)
print("HUẤN LUYỆN MOBILENETV2")
print("="*50)

# Train model
mobilenet_history = mobilenet_model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=get_callbacks('mobilenet'),
    verbose=1
)

In [ ]:
# Fine-tuning: Mở khóa một số layers cuối của base model
print("\n" + "="*50)
print("FINE-TUNING MOBILENETV2")
print("="*50)

# Mở khóa 30 layers cuối
mobilenet_base.trainable = True
for layer in mobilenet_base.layers[:-30]:
    layer.trainable = False

# Compile lại với learning rate thấp hơn
mobilenet_model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Learning rate thấp cho fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune training
mobilenet_finetune_history = mobilenet_model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=get_callbacks('mobilenet_finetuned'),
    verbose=1
)

## 8. Đánh giá mô hình

In [ ]:
def plot_training_history(history, title='Training History'):
    """Vẽ biểu đồ quá trình training"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Accuracy
    axes[0].plot(history.history['accuracy'], label='Training', linewidth=2)
    axes[0].plot(history.history['val_accuracy'], label='Validation', linewidth=2)
    axes[0].set_title('Độ chính xác (Accuracy)', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Loss
    axes[1].plot(history.history['loss'], label='Training', linewidth=2)
    axes[1].plot(history.history['val_loss'], label='Validation', linewidth=2)
    axes[1].set_title('Hàm mất mát (Loss)', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.suptitle(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, f'{title.replace(" ", "_").lower()}.png'), dpi=150)
    plt.show()

# Vẽ training history
plot_training_history(mobilenet_history, 'MobileNetV2 Training History')

In [ ]:
def evaluate_model(model, val_gen, model_name='Model'):
    """Đánh giá model trên tập validation"""
    val_gen.reset()
    
    # Dự đoán
    predictions = model.predict(val_gen, verbose=1)
    y_pred = np.argmax(predictions, axis=1)
    y_true = val_gen.classes
    
    # Tính accuracy
    acc = accuracy_score(y_true, y_pred)
    
    print("\n" + "="*50)
    print(f"ĐÁNH GIÁ {model_name.upper()}")
    print("="*50)
    print(f"\nĐộ chính xác tổng: {acc:.4f} ({acc*100:.2f}%)")
    
    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=CLASSES))
    
    return y_true, y_pred, predictions, acc

# Đánh giá MobileNet
y_true, y_pred, predictions, accuracy = evaluate_model(mobilenet_model, val_generator, 'MobileNetV2')

In [ ]:
def plot_confusion_matrix(y_true, y_pred, title='Confusion Matrix'):
    """Vẽ ma trận nhầm lẫn"""
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        cm, 
        annot=True, 
        fmt='d', 
        cmap='Blues',
        xticklabels=CLASSES, 
        yticklabels=CLASSES,
        annot_kws={"size": 14}
    )
    plt.title(f'{title}\n(Ma trận nhầm lẫn)', fontsize=14, fontweight='bold')
    plt.xlabel('Dự đoán', fontsize=12)
    plt.ylabel('Thực tế', fontsize=12)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'confusion_matrix.png'), dpi=150)
    plt.show()
    
    # In chi tiết
    print("\nChi tiết ma trận nhầm lẫn:")
    for i, cls in enumerate(CLASSES):
        correct = cm[i, i]
        total = sum(cm[i, :])
        print(f"  {cls}: {correct}/{total} ({correct/total*100:.1f}%)")

plot_confusion_matrix(y_true, y_pred, 'MobileNetV2')

## 9. Lưu mô hình

In [ ]:
# Lưu model cuối cùng
model_save_path = os.path.join(MODEL_DIR, 'mobilenet_final.keras')
mobilenet_model.save(model_save_path)
print(f"Đã lưu model: {model_save_path}")

# Lưu weights riêng
weights_save_path = os.path.join(MODEL_DIR, 'mobilenet_weights.weights.h5')
mobilenet_model.save_weights(weights_save_path)
print(f"Đã lưu weights: {weights_save_path}")

## 10. Dự đoán ảnh mới

In [ ]:
def load_and_preprocess_image(img_path):
    """Load và tiền xử lý ảnh để dự đoán"""
    img = Image.open(img_path)
    img = img.convert('RGB')
    img = img.resize(IMG_SIZE)
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array, img

def predict_image(model, img_path):
    """Dự đoán một ảnh"""
    img_array, img = load_and_preprocess_image(img_path)
    
    # Dự đoán
    pred = model.predict(img_array, verbose=0)
    class_idx = np.argmax(pred[0])
    confidence = pred[0][class_idx]
    class_name = CLASSES[class_idx]
    
    return class_name, confidence, pred[0], img

def visualize_prediction(model, img_path):
    """Hiển thị kết quả dự đoán với biểu đồ"""
    class_name, confidence, probs, img = predict_image(model, img_path)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Hiển thị ảnh
    axes[0].imshow(img)
    axes[0].set_title(f'Dự đoán: {class_name}\n({CLASS_NAMES_VN[class_name]})\nĐộ tin cậy: {confidence:.2%}', 
                     fontsize=12, fontweight='bold')
    axes[0].axis('off')
    
    # Biểu đồ xác suất
    colors = ['#2ecc71' if i == np.argmax(probs) else '#95a5a6' for i in range(len(CLASSES))]
    bars = axes[1].barh(CLASSES, probs, color=colors)
    axes[1].set_xlim(0, 1)
    axes[1].set_title('Xác suất các loại bệnh', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Xác suất')
    
    # Thêm giá trị trên mỗi bar
    for bar, prob in zip(bars, probs):
        axes[1].text(prob + 0.02, bar.get_y() + bar.get_height()/2, 
                    f'{prob:.2%}', va='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    return class_name, confidence

print("Hàm dự đoán đã sẵn sàng!")

In [ ]:
# Test dự đoán với một số ảnh từ validation set
import random

print("="*50)
print("TEST DỰ ĐOÁN")
print("="*50)

# Lấy ngẫu nhiên 3 ảnh từ mỗi class
for cls in CLASSES:
    cls_path = os.path.join(VAL_DIR, cls)
    if os.path.exists(cls_path):
        imgs = [f for f in os.listdir(cls_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
        sample_img = random.choice(imgs)
        img_path = os.path.join(cls_path, sample_img)
        
        print(f"\nẢnh thực tế: {cls} ({CLASS_NAMES_VN[cls]})")
        pred_class, conf = visualize_prediction(mobilenet_model, img_path)
        
        if pred_class == cls:
            print("✅ Dự đoán ĐÚNG!")
        else:
            print(f"❌ Dự đoán SAI! (Dự đoán: {pred_class})")

## 11. Tổng kết và Báo cáo

In [ ]:
print("="*60)
print("                    TỔNG KẾT                    ")
print("="*60)

print(f"""
📊 THÔNG TIN DỮ LIỆU:
   - Training set: {train_generator.samples} ảnh
   - Validation set: {val_generator.samples} ảnh
   - Số class: {NUM_CLASSES}
   - Classes: {CLASSES}

🔧 KỸ THUẬT CHỐNG OVERFITTING:
   ✅ Data Augmentation mạnh (rotation, zoom, flip, shift)
   ✅ Dropout layers (0.25 - 0.5)
   ✅ Batch Normalization
   ✅ L2 Regularization (0.001)
   ✅ Early Stopping (patience=15)
   ✅ Learning Rate Scheduling
   ✅ Global Average Pooling
   ✅ Transfer Learning + Fine-tuning

📈 KẾT QUẢ:
   - Model: MobileNetV2 (Transfer Learning)
   - Validation Accuracy: {accuracy*100:.2f}%

💾 FILES ĐÃ LƯU:
   - Model: {model_save_path}
   - Weights: {weights_save_path}
   - Biểu đồ: {OUTPUT_DIR}/
""")

print("="*60)
print("               HOÀN THÀNH!               ")
print("="*60)

## 12. Hướng dẫn sử dụng model đã train

```python
# Load model
from tensorflow.keras.models import load_model
model = load_model('models/mobilenet_final.keras')

# Dự đoán
class_name, confidence = predict_image(model, 'path/to/your/image.jpg')
print(f"Kết quả: {class_name} - Độ tin cậy: {confidence:.2%}")
```